## 기본 라이브러리 불러오기

In [2]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

## 기본 데이터 불러오기

### 서울특별시 공공자전거 대여이력 정보

In [12]:
start_time = time.time()
#_columns = ['자전거번호', '대여일자', '대여시간차', '대여대여소번호', '반납대여소번호']
df_BikeRent_2023 = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_2023_preprocessed.parquet')#, columns = _columns)
print("read_csv time :", time.time() - start_time)

read_csv time : 66.51031017303467


In [14]:
df_BikeRent_2023.shape[0]

45363848

In [16]:
df_BikeRent_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45363848 entries, 0 to 46388222
Data columns (total 16 columns):
 #   Column      Dtype          
---  ------      -----          
 0   자전거번호       object         
 1   대여대여소번호     object         
 2   대여대여소명      object         
 3   대여거치대       int64          
 4   반납대여소번호     object         
 5   반납대여소명      object         
 6   반납거치대       object         
 7   이용시간(분)     int64          
 8   이용거리(M)     float64        
 9   대여일자        object         
 10  대여시간        object         
 11  반납일자        object         
 12  반납시간        object         
 13  대여시간차       timedelta64[ns]
 14  이용시간-대여시간차  float64        
 15  이용속도(km/h)  float64        
dtypes: float64(3), int64(2), object(10), timedelta64[ns](1)
memory usage: 5.7+ GB


In [18]:
df_BikeRent_2023.head()

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
0,SPB-39194,01554,번동사거리,0,01554,번동사거리,0,0,0.00,2023-01-01,00:02:13,2023-01-01,00:02:43,0 days 00:00:30,-0.500000,NaN
1,SPB-43457,01653,노원역1번출구,0,04003,상계초교 입구 교차로 명주빌딩 앞,0,2,0.00,2023-01-01,00:01:20,2023-01-01,00:03:28,0 days 00:02:08,-0.133333,0.00000
2,SPB-44383,02601,석촌호수 아뜰리에,0,02601,석촌호수 아뜰리에,0,0,0.00,2023-01-01,00:04:01,2023-01-01,00:04:21,0 days 00:00:20,-0.333333,NaN
3,SPB-37186,01080,엘웨딩,0,03668,성내삼성아파트,0,2,0.00,2023-01-01,00:02:09,2023-01-01,00:04:59,0 days 00:02:50,-0.833333,0.00000
5,SPB-35099,02038,동작역 7번출구,0,04178,이수성결교회 옆,0,4,1005.05,2023-01-01,00:01:24,2023-01-01,00:05:37,0 days 00:04:13,-0.216667,15.07575


In [32]:
# 1. '대여일자' 컬럼을 object형에서 datetime형으로 변환
df_BikeRent_2023['대여일자'] = pd.to_datetime(df_BikeRent_2023['대여일자'])

# 2. 새로운 컬럼으로 년, 월, 일 생성
df_BikeRent_2023['대여_년'] = df_BikeRent_2023['대여일자'].dt.year
df_BikeRent_2023['대여_월'] = df_BikeRent_2023['대여일자'].dt.month
df_BikeRent_2023['대여_일'] = df_BikeRent_2023['대여일자'].dt.day

In [34]:
df_BikeRent_2023.head(2)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h),대여_년,대여_월,대여_일
0,SPB-39194,01554,번동사거리,0,01554,번동사거리,0,0,0.0,2023-01-01,00:02:13,2023-01-01,00:02:43,0 days 00:00:30,-0.500000,NaN,2023,1,1
1,SPB-43457,01653,노원역1번출구,0,04003,상계초교 입구 교차로 명주빌딩 앞,0,2,0.0,2023-01-01,00:01:20,2023-01-01,00:03:28,0 days 00:02:08,-0.133333,0.0,2023,1,1


In [36]:
grouped_data = df_BikeRent_2023.groupby(['자전거번호', '대여_월']).agg({
    '이용거리(M)': ['sum', 'mean'],             # 이용거리의 합계
    '이용시간(분)': ['sum', 'mean'],           # 이용시간의 평균
    '대여일자': ['count']           # 이용속도의 최대값
})

In [38]:
grouped_data.shape[0]

427488

In [42]:
grouped_data.reset_index().head()

자전거번호 대여_월    이용거리(M)              이용시간(분)             대여일자
                         sum         mean     sum       mean count
0  SPB-30001    1  201289.10  1597.532540    1774  14.079365   126
1  SPB-30001    2  261989.24  2360.263423    2254  20.306306   111
2  SPB-30001    3  468359.63  2401.844256    3745  19.205128   195
3  SPB-30001    4  409644.01  3938.884712    3575  34.375000   104
4  SPB-30001    5  315511.67  2607.534463    2508  20.727273   121

In [48]:
pivot_df = grouped_data.pivot_table(index='자전거번호', columns='대여_월', values=['이용거리(M)', '이용시간(분)'], aggfunc='sum')

In [50]:
pivot_df.head()

이용거리(M)                                                      \
                  mean                                                       
대여_월                1            2            3            4            5    
자전거번호                                                                        
SPB-30001  1597.532540  2360.263423  2401.844256  3938.884712  2607.534463   
SPB-30006  1468.723636  1717.237500  1920.765200  1798.489065  2225.908141   
SPB-30008  1222.382143  1878.965714  2277.800000  1902.278434  2182.769006   
SPB-30009          NaN  2058.221429  1763.318829  2071.568488  3322.989000   
SPB-30010  1336.259538  1630.415913  1733.082848  1947.733293  1720.982454   

                                                                            \
                                                                             
대여_월                6            7            8            9            10   
자전거번호                                                                        
SPB-30001  2265.619877  2449.579078  1754.730588  2441.748571  2058.824472   
SPB-30006  2427.211295  2482.220149  1585.403357  2791.540560  2172.033459   
SPB-30008  2227.949083  2630.634406  3092.678212  2912.774914  2275.691524   
SPB-30009  2160.264167  1741.867347  1863.074444  2127.794615  1845.711702   
SPB-30010  2153.064017  1968.328874  1885.984510  2117.909704  1928.055867   

           ... 이용시간(분)                                                  \
           ...     sum                                                   
대여_월       ...      3       4       5       6       7       8       9    
자전거번호      ...                                                           
SPB-30001  ...  3745.0  3575.0  2508.0  3031.0  3057.0  1800.0  3682.0   
SPB-30006  ...  2771.0  2789.0  3739.0  4288.0  4336.0  2506.0  3133.0   
SPB-30008  ...   367.0  1181.0  2988.0  1930.0  2992.0  3994.0  2624.0   
SPB-30009  ...  2069.0  3100.0   642.0   725.0   678.0  2938.0  1747.0   
SPB-30010  ...  2566.0  3140.0  3308.0  4352.0  2541.0  1501.0  2146.0   

                                   
                                   
대여_월           10      11      12  
자전거번호                              
SPB-30001  2410.0  2172.0   925.0  
SPB-30006  3084.0   603.0  1258.0  
SPB-30008  2219.0  1858.0   827.0  
SPB-30009  2229.0     NaN     NaN  
SPB-30010  2408.0   642.0   580.0  

[5 rows x 48 columns]

In [52]:
pivot_df.columns

MultiIndex([('이용거리(M)', 'mean',  1),
            ('이용거리(M)', 'mean',  2),
            ('이용거리(M)', 'mean',  3),
            ('이용거리(M)', 'mean',  4),
            ('이용거리(M)', 'mean',  5),
            ('이용거리(M)', 'mean',  6),
            ('이용거리(M)', 'mean',  7),
            ('이용거리(M)', 'mean',  8),
            ('이용거리(M)', 'mean',  9),
            ('이용거리(M)', 'mean', 10),
            ('이용거리(M)', 'mean', 11),
            ('이용거리(M)', 'mean', 12),
            ('이용거리(M)',  'sum',  1),
            ('이용거리(M)',  'sum',  2),
            ('이용거리(M)',  'sum',  3),
            ('이용거리(M)',  'sum',  4),
            ('이용거리(M)',  'sum',  5),
            ('이용거리(M)',  'sum',  6),
            ('이용거리(M)',  'sum',  7),
            ('이용거리(M)',  'sum',  8),
            ('이용거리(M)',  'sum',  9),
            ('이용거리(M)',  'sum', 10),
            ('이용거리(M)',  'sum', 11),
            ('이용거리(M)',  'sum', 12),
            ('이용시간(분)', 'mean',  1),
            ('이용시간(분)', 'mean',  2),
            ('이용시간(분)', 'mean',  3),
 

In [44]:
grouped_data.columns

MultiIndex([('이용거리(M)',   'sum'),
            ('이용거리(M)',  'mean'),
            ('이용시간(분)',   'sum'),
            ('이용시간(분)',  'mean'),
            (   '대여일자', 'count')],
           )